# Benchmarking

## Helper Functions

In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from time import time

# commonroad
from commonroad.geometry.shape import Polygon, ShapeGroup, Shape, Rectangle
from commonroad.scenario.lanelet import LaneletNetwork
from commonroad.scenario.obstacle import StaticObstacle, ObstacleType
from commonroad.scenario.scenario import Scenario
from commonroad.scenario.trajectory import State, Trajectory
from commonroad.prediction.prediction import TrajectoryPrediction, SetBasedPrediction

#commonroad-io
from commonroad.common.file_reader import CommonRoadFileReader

# commonroad_dc
import commonroad_dc.pycrcc as pycrcc
from commonroad_dc.collision.collision_detection.pycrcc_collision_dispatch import create_collision_checker, create_collision_object
from commonroad_dc.collision.visualization.draw_dispatch import draw_object
from commonroad_dc.boundary import boundary
from commonroad_dc.collision.trajectory_queries import trajectory_queries
from commonroad_dc.pycrcc.Util import trajectory_enclosure_polygons_static as trajectory_check_lane_polygon_enclosure
import commonroad_dc.pycrcc.Util as Util


def load_waypoints(scenario_name):

    try:
        data=np.load("waypoints/"+scenario_name+"_waypoints.npz")
    except:
        return None
    
    loaded={"car_shape": data["car_shape"], "trajectories": data["trajectories"], "start_time_step": data["start_time_step"]}

    data.close()
    
    trajectory_list=list()

    start_time_step=loaded["start_time_step"]
    
    for ind,ts_start_idx in enumerate(start_time_step):
        tvobstacle=pycrcc.TimeVariantCollisionObject(int(ts_start_idx))
        car_half_length,car_half_width=loaded["car_shape"]/2
        for state in loaded["trajectories"][ind]:
            tvobstacle.append_obstacle(pycrcc.RectOBB(car_half_length, car_half_width, state[2], state[0], state[1]))
        
        tvobstacle, err=trajectory_queries.trajectory_preprocess_obb_sum(tvobstacle)
    
        trajectory_list.append(tvobstacle)
    
    return trajectory_list, loaded

def scenario_init(scenario_name, directory):
    
    def open_scenario(scenario_filename):
        crfr = CommonRoadFileReader(scenario_filename)
        scenario, planning_problem_set = crfr.open()
        return scenario, planning_problem_set
    
    traj_ret = load_waypoints(scenario_name)
    if traj_ret is None:
        return None
    
    trajectories, loaded = traj_ret
    
    waypoints=loaded["trajectories"]
    
    car_shape=loaded["car_shape"]
    car_half_length, car_half_width = (car_shape/2)
    
    cur_time_1=time()
    
    scenario, planning_problem_set = open_scenario(directory + scenario_name + '.xml')
    
    cur_time_2=time()
    
    cur_time_1 = time()
    road_boundary_sg_triangles = boundary.create_road_boundary_obstacle(scenario,method='triangulation', return_scenario_obstacle=False)
    cur_time_2 = time()
    time_triangles = cur_time_2-cur_time_1
    
    cur_time_1 = time()
    road_boundary_sg_aligned_triangles = boundary.create_road_boundary_obstacle(scenario, method='aligned_triangulation', return_scenario_obstacle=False, axis='auto')
    cur_time_2 = time()
    time_aligned_triangles = cur_time_2-cur_time_1
    
    cur_time_1 = time()
    road_boundary_sg_rectangles = boundary.create_road_boundary_obstacle(scenario, method='obb_rectangles', return_scenario_obstacle=False)
    cur_time_2 = time()
    time_rectangles = cur_time_2-cur_time_1
    
    cur_time_1 = time()

    road_boundary_sg_polygons_grid = boundary.create_road_polygons(scenario, method='whole_polygon_tiled', max_cell_width=car_half_length*6, max_cell_height=car_half_length*20,triangulate=False)

    cur_time_2 = time()
    time_polygons_grid = cur_time_2-cur_time_1
    
    cur_time_1 = time()
    road_boundary_sg_polygons_lanes = boundary.create_road_polygons(scenario, method='lane_polygons', buffer=1,resample=1, triangulate=False)
    cur_time_2 = time()
    time_polygons_lanes = cur_time_2-cur_time_1
    
    cur_time_1 = time()
    road_boundary_sg_polygons_whole = boundary.create_road_polygons(scenario, method='whole_polygon', triangulate=False)
    cur_time_2 = time()
    time_polygons_whole = cur_time_2-cur_time_1   
    
    dyn_obstacles=get_scenario_dynamic_obstacles(scenario)
    
    road_boundaries, road_polygons, construction_times={},{},{}
    road_boundaries['triangles']=road_boundary_sg_triangles
    road_boundaries['aligned_triangles']=road_boundary_sg_aligned_triangles
    road_boundaries['rectangles']=road_boundary_sg_rectangles
    
    road_polygons['polygons_lanes']=road_boundary_sg_polygons_lanes
    road_polygons['whole_polygons']=road_boundary_sg_polygons_whole
    road_polygons['whole_polygons_tiled']=road_boundary_sg_polygons_grid
    
    construction_times['triangles']=time_triangles
    construction_times['aligned_triangles']=time_aligned_triangles
    construction_times['rectangles']=time_rectangles
    construction_times['polygons_lanes']=time_polygons_lanes
    construction_times['whole_polygons']=time_polygons_whole
    construction_times['whole_polygons_tiled']=time_polygons_grid
    
    return scenario, dyn_obstacles, waypoints, trajectories, car_shape, road_boundaries, road_polygons, construction_times

def get_scenario_dynamic_obstacles(scenario):
    dyn_obstacles_list=list()

    for dyn_obst in scenario.dynamic_obstacles:
        if isinstance(dyn_obst.prediction,TrajectoryPrediction):
            co=create_collision_object(dyn_obst.prediction)
            co, err=trajectory_queries.trajectory_preprocess_obb_sum(co)
            if err:
                print('OBB-hull preprocessing is implemented only for trajectories containing one OBB or AABB box per time step')
            dyn_obstacles_list.append(co)
        else:
            # collision checks with SetBasedPrediction are supported but not implemented in this Jupyter notebook
            return list()

    return dyn_obstacles_list

def measure_time_triangles(waypoints, trajectories, car_shape, road_boundary_triangles, road_polygons, use_fcl_narrowphase=False, num_iter=1):

    candidate_trajectories=list()
    
    
    car_half_length, car_half_width=(car_shape/2)
    
    Util.perfStart()
    
    cur_time_1=time()
    
    for i in range(num_iter):
        for ind,states in enumerate(waypoints):
            if trajectory_check_lane_polygon_enclosure(road_polygons, car_half_length, car_half_width, states[0:0])==-1:    
                candidate_trajectories.append(trajectories[ind])
    result=trajectory_queries.trajectories_collision_static_obstacles_grid(candidate_trajectories, road_boundary_triangles, num_cells=32, auto_orientation=True, optimize_triangles=(use_fcl_narrowphase==False))
    
    cur_time_2 = time()
    
    Util.perfReport()
    
    num_noncol=(np.asarray(result)==-1).sum()
    num_col=len(candidate_trajectories)-num_noncol
        
    # return values: time to check all trajectories for collisions; number of colliding trajectories, number of collision-free trajectories
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol

def measure_time_rectangles(waypoints, trajectories, car_shape, road_boundary_rectangles, road_polygons, num_iter=1):
    
    candidate_trajectories=list()
    
    car_half_length, car_half_width=(car_shape/2)
    
    Util.perfStart()
    
    cur_time_1=time()
    for i in range(num_iter):
        for ind,states in enumerate(waypoints):
            if trajectory_check_lane_polygon_enclosure(road_polygons,car_half_length, car_half_width, states[0:0])==-1:    
                candidate_trajectories.append(trajectories[ind])
    
    result=trajectory_queries.trajectories_collision_static_obstacles_grid(candidate_trajectories, road_boundary_rectangles, num_cells=32, auto_orientation=True, optimize_triangles=False)

    cur_time_2 = time()

    Util.perfReport()
    
    num_noncol=(np.asarray(result)==-1).sum()
    num_col=len(candidate_trajectories)-num_noncol
        
    # return values: time to check all trajectories for collisions; number of colliding trajectories, number of collision-free trajectories
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol


def measure_time_rectangles_fcl(waypoints, trajectories, car_shape, road_boundary_rectangles, road_polygons, num_iter=1, method=[0,32,1000]):
    
    candidate_trajectories=list()
    
    car_half_length, car_half_width=(car_shape/2)
    
    Util.perfStart()
    
    cur_time_1=time()
    for i in range(num_iter):
        for ind,states in enumerate(waypoints):
            if trajectory_check_lane_polygon_enclosure(road_polygons,car_half_length, car_half_width, states[0:0])==-1:    
                candidate_trajectories.append(trajectories[ind])
    
    result=trajectory_queries.trajectories_collision_static_obstacles_fcl(candidate_trajectories, road_boundary_rectangles, *method)

    cur_time_2 = time()

    Util.perfReport()
    
    num_noncol=(np.asarray(result)==-1).sum()
    num_col=len(candidate_trajectories)-num_noncol
        
    # return values: time to check all trajectories for collisions; number of colliding trajectories, number of collision-free trajectories
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol


    
def measure_time_rectangles_box2d(waypoints, trajectories, car_shape, road_boundary_rectangles, road_polygons, num_iter=1):
    
    candidate_trajectories=list()
    
    car_half_length, car_half_width=(car_shape/2)
    
    Util.perfStart()
    
    cur_time_1=time()
    for i in range(num_iter):
        for ind,states in enumerate(waypoints):
            if trajectory_check_lane_polygon_enclosure(road_polygons,car_half_length, car_half_width, states[0:0])==-1:    
                candidate_trajectories.append(trajectories[ind])
    
    result=trajectory_queries.trajectories_collision_static_obstacles_box2d(candidate_trajectories, road_boundary_rectangles)

    cur_time_2 = time()

    Util.perfReport()
    
    num_noncol=(np.asarray(result)==-1).sum()
    num_col=len(candidate_trajectories)-num_noncol
        
    # return values: time to check all trajectories for collisions; number of colliding trajectories, number of collision-free trajectories
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol
    
def measure_time_polygons(trajectories, road_polygons, num_iter=1):

    Util.perfStart()
    cur_time_1=time()
    for i in range(num_iter):
        result=trajectory_queries.trajectories_enclosure_polygons_static_grid(trajectories, road_polygons, num_cells=32)

    cur_time_2 = time()
    Util.perfReport()

    num_noncol=(np.asarray(result)==-1).sum()
    num_col=len(trajectories)-num_noncol
    
    # return values: time to check all trajectories for collisions; number of colliding trajectories, number of collision-free trajectories   
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol


def measure_time_dynamic_fcl(scenario, trajectories, dynamic_obstacles, method=[0,32,1000], num_iter=1):
    cur_time_1=time()
    for i in range(num_iter):
        res_dynamic=trajectory_queries.trajectories_collision_dynamic_obstacles_fcl(trajectories, dynamic_obstacles, *method)
    cur_time_2 = time()
    
    num_noncol=(np.asarray(res_dynamic)==-1).sum()
    num_col=len(trajectories)-num_noncol
    
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol

def measure_time_dynamic_grid(scenario, trajectories,dynamic_obstacles, num_iter=1):
    cur_time_1=time()
    for i in range(num_iter):
        res_dynamic = trajectory_queries.trajectories_collision_dynamic_obstacles_grid(trajectories, dynamic_obstacles, num_cells=32)
    cur_time_2 = time()
    
    num_noncol=(np.asarray(res_dynamic)==-1).sum()
    num_col=len(trajectories)-num_noncol
    
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol

def measure_time_dynamic_box2d(scenario, trajectories,dynamic_obstacles, num_iter=1):
    cur_time_1=time()
    for i in range(num_iter):
        res_dynamic = trajectory_queries.trajectories_collision_dynamic_obstacles_box2d(trajectories, dynamic_obstacles)
    cur_time_2 = time()
    
    num_noncol=(np.asarray(res_dynamic)==-1).sum()
    num_col=len(trajectories)-num_noncol
    
    return (cur_time_2-cur_time_1)/num_iter,num_col,num_noncol

directory = 'scenarios/'

In [11]:
import zipfile
with zipfile.ZipFile(directory+'scenarios.zip', 'r') as zip_ref:
    zip_ref.extractall(directory)

## 1 Benchmark Pre-Processing and Road Compliance Checks

In [12]:
import os

export_res = dict()
export_ct = dict()
export_obst_num = dict()
for ind, el in enumerate(os.listdir(directory)):
    # open scenario
    scenario_name = os.path.splitext(el)[0]
    
    # pre-processes scenario for all different methods for road compliance checking
    init_list = scenario_init(scenario_name, directory)
    if init_list is None:
        pass
    else:
        print(str(ind) + ' ' + scenario_name)
        #print(scenario_name)
        #continue
        # extract all return values
        scenario, dyn_obstacles, waypoints, trajectories, car_shape, road_boundaries, road_polygons, construction_times = init_list
        
        # time measurements for pre-processing for road-compliance check, e.g., construction of road boundaries
        print("Pre-processing time: triangulation: {}, axis-aligned triangulation: {}, oriented rectangles: {}, occupancy inclusion (polygon tiles): {}".format(construction_times['triangles'],construction_times['aligned_triangles'],construction_times['rectangles'],construction_times['whole_polygons_tiled']))
        export_ct[scenario_name] = (construction_times['triangles'], construction_times['aligned_triangles'], construction_times['rectangles'], construction_times['whole_polygons_tiled'], construction_times['polygons_lanes'], construction_times['whole_polygons'])
        print("Number of objects: triangulation: {}, axis-aligned triangulation: {}, oriented rectangles: {}, occupancy inclusion (polygon tiles): {}".format(road_boundaries['triangles'].size(), road_boundaries['aligned_triangles'].size(), road_boundaries['rectangles'].size(), road_polygons['whole_polygons_tiled'].size()))
        export_obst_num[scenario_name] = (road_boundaries['triangles'].size(), road_boundaries['aligned_triangles'].size(), road_boundaries['rectangles'].size(), road_polygons['whole_polygons_tiled'].size(), road_polygons['polygons_lanes'].size(), road_polygons['whole_polygons'].size() )
        
        num_iter=1
        
        # time measurements for road compliance check with multiple trajectories
        res_tri = measure_time_triangles(waypoints, trajectories, car_shape, road_boundaries['triangles'], road_polygons['whole_polygons_tiled'], use_fcl_narrowphase=False, num_iter=num_iter)
        res_aligned_tri = measure_time_triangles(waypoints, trajectories, car_shape, road_boundaries['aligned_triangles'], road_polygons['whole_polygons_tiled'], use_fcl_narrowphase=False, num_iter=num_iter)
        res_rect = measure_time_rectangles(waypoints, trajectories, car_shape, road_boundaries['rectangles'], road_polygons['whole_polygons_tiled'], num_iter=num_iter)
        res_rect_box2d = measure_time_rectangles_box2d(waypoints, trajectories, car_shape, road_boundaries['rectangles'], road_polygons['whole_polygons_tiled'], num_iter=num_iter)
        res_rect_fcl_btree = measure_time_rectangles_fcl(waypoints, trajectories, car_shape, road_boundaries['rectangles'], road_polygons['whole_polygons_tiled'],method=[0,32,1000], num_iter=num_iter)
        res_rect_fcl_sh = measure_time_rectangles_fcl(waypoints, trajectories, car_shape, road_boundaries['rectangles'], road_polygons['whole_polygons_tiled'],method=[1,32,1000], num_iter=num_iter)
        res_rect_fcl_it = measure_time_rectangles_fcl(waypoints, trajectories, car_shape, road_boundaries['rectangles'], road_polygons['whole_polygons_tiled'],method=[2,32,1000], num_iter=num_iter)
        res_rect_fcl_sap = measure_time_rectangles_fcl(waypoints, trajectories, car_shape, road_boundaries['rectangles'], road_polygons['whole_polygons_tiled'],method=[4,32,1000], num_iter=num_iter)    
        
        res_poly_tiles = measure_time_polygons(trajectories, road_polygons['whole_polygons_tiled'], num_iter=num_iter)
        res_poly_lanes = measure_time_polygons(trajectories, road_polygons['polygons_lanes'], num_iter=num_iter)
        res_poly_whole = measure_time_polygons(trajectories, road_polygons['whole_polygons'], num_iter=num_iter)
        
        print("Triangulation, Uniform grid broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_tri[0], res_tri[1], res_tri[2]))
    
        print("Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_aligned_tri[0], res_aligned_tri[1], res_aligned_tri[2]))
        
        print("Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_rect[0], res_rect[1], res_rect[2]))
        print("Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_rect_box2d[0], res_rect_box2d[1], res_rect_box2d[2]))
        print("Oriented Rectangles, FCL dynamic AABB tree broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_rect_fcl_btree[0], res_rect_fcl_btree[1], res_rect_fcl_btree[2]))
        print("Oriented Rectangles, FCL spatial hashing broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_rect_fcl_sh[0], res_rect_fcl_sh[1], res_rect_fcl_sh[2]))
        print("Oriented Rectangles, FCL interval tree broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_rect_fcl_it[0], res_rect_fcl_it[1], res_rect_fcl_it[2]))
        print("Oriented Rectangles, FCL Sweep-and-Prune broadphase; time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_rect_fcl_sap[0], res_rect_fcl_sap[1], res_rect_fcl_sap[2]))
        
        print("Occupancy Inclusion (polygon tiles); time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_poly_tiles[0], res_poly_tiles[1], res_poly_tiles[2]))
        print("Occupancy Inclusion (lanes); time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_poly_lanes[0], res_poly_lanes[1], res_poly_lanes[2]))
        print("Occupancy Inclusion (whole polygon); time to check all trajectories: {}, number of colliding trajectories: {}, number of collision-free trajectories: {}".format(res_poly_whole[0], res_poly_whole[1], res_poly_whole[2]))
        
                
        export_res[scenario_name] = (res_tri, res_aligned_tri, res_rect, res_rect_box2d, res_rect_fcl_btree, res_rect_fcl_sh, res_rect_fcl_it, res_rect_fcl_sap, res_poly_tiles, res_poly_lanes, res_poly_whole )
        

0 USA_Peach-2_1_T-1
Pre-processing time: triangulation: 0.0657815933227539, axis-aligned triangulation: 0.00506281852722168, oriented rectangles: 0.0024826526641845703, occupancy inclusion (polygon tiles): 0.0038301944732666016
Number of objects: triangulation: 449, axis-aligned triangulation: 447, oriented rectangles: 141, occupancy inclusion (polygon tiles): 17
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.01875925064086914, number of colliding trajectories: 21, number of collision-free trajectories: 979
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.016638755798339844, number of colliding trajectories: 21, number of collision-free trajectories: 979
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.011028289794921875, number of colliding trajectories: 21, number of collision-free trajectories: 979
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: 0.

5 CHN_Sha-13_2_T-1
Pre-processing time: triangulation: 0.05724525451660156, axis-aligned triangulation: 0.005352973937988281, oriented rectangles: 0.0017740726470947266, occupancy inclusion (polygon tiles): 0.010041475296020508
Number of objects: triangulation: 618, axis-aligned triangulation: 553, oriented rectangles: 180, occupancy inclusion (polygon tiles): 46
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.020026206970214844, number of colliding trajectories: 1000, number of collision-free trajectories: 0
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.021368026733398438, number of colliding trajectories: 1000, number of collision-free trajectories: 0
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.011307001113891602, number of colliding trajectories: 1000, number of collision-free trajectories: 0
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: 0

9 CHN_Sha-1_4_T-1
Pre-processing time: triangulation: 0.05845284461975098, axis-aligned triangulation: 0.005392313003540039, oriented rectangles: 0.0018155574798583984, occupancy inclusion (polygon tiles): 0.010008096694946289
Number of objects: triangulation: 618, axis-aligned triangulation: 553, oriented rectangles: 180, occupancy inclusion (polygon tiles): 46
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.035151004791259766, number of colliding trajectories: 148, number of collision-free trajectories: 852
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.029356718063354492, number of colliding trajectories: 148, number of collision-free trajectories: 852
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.017396926879882812, number of colliding trajectories: 148, number of collision-free trajectories: 852
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories:

13 RUS_Bicycle-11_1_T-1
Pre-processing time: triangulation: 0.07487010955810547, axis-aligned triangulation: 0.0012614727020263672, oriented rectangles: 0.0005512237548828125, occupancy inclusion (polygon tiles): 0.0008392333984375
Number of objects: triangulation: 422, axis-aligned triangulation: 108, oriented rectangles: 6, occupancy inclusion (polygon tiles): 3
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.01275634765625, number of colliding trajectories: 134, number of collision-free trajectories: 866
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.010252237319946289, number of colliding trajectories: 134, number of collision-free trajectories: 866
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.009882926940917969, number of colliding trajectories: 134, number of collision-free trajectories: 866
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: 0

17 CHN_Sha-2_1_T-1
Pre-processing time: triangulation: 0.581425666809082, axis-aligned triangulation: 0.04399442672729492, oriented rectangles: 0.03710007667541504, occupancy inclusion (polygon tiles): 0.1267075538635254
Number of objects: triangulation: 2130, axis-aligned triangulation: 1517, oriented rectangles: 609, occupancy inclusion (polygon tiles): 192
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.020613431930541992, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.021953105926513672, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.013816118240356445, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: 0.014

...

62 RUS_Bicycle-3_1_T-1
Pre-processing time: triangulation: 0.1480545997619629, axis-aligned triangulation: 0.0023796558380126953, oriented rectangles: 0.0010194778442382812, occupancy inclusion (polygon tiles): 0.0011937618255615234
Number of objects: triangulation: 509, axis-aligned triangulation: 116, oriented rectangles: 10, occupancy inclusion (polygon tiles): 3
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.012877702713012695, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.010527610778808594, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.010096549987792969, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories

66 CHN_Sha-2_8_T-1
Pre-processing time: triangulation: 0.5793125629425049, axis-aligned triangulation: 0.043441057205200195, oriented rectangles: 0.036956071853637695, occupancy inclusion (polygon tiles): 0.12542939186096191
Number of objects: triangulation: 2130, axis-aligned triangulation: 1517, oriented rectangles: 609, occupancy inclusion (polygon tiles): 192
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.020520448684692383, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.021035194396972656, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.013769149780273438, number of colliding trajectories: 0, number of collision-free trajectories: 1000
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: 0

70 USA_Lanker-2_18_T-1
Pre-processing time: triangulation: 0.09654831886291504, axis-aligned triangulation: 0.007890939712524414, oriented rectangles: 0.005014181137084961, occupancy inclusion (polygon tiles): 0.00850534439086914
Number of objects: triangulation: 572, axis-aligned triangulation: 619, oriented rectangles: 199, occupancy inclusion (polygon tiles): 44
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.011860847473144531, number of colliding trajectories: 875, number of collision-free trajectories: 125
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.013207674026489258, number of colliding trajectories: 875, number of collision-free trajectories: 125
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.009461402893066406, number of colliding trajectories: 875, number of collision-free trajectories: 125
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectori

74 DEU_Muc-11_1_T-1
Pre-processing time: triangulation: 0.039115190505981445, axis-aligned triangulation: 0.002866506576538086, oriented rectangles: 0.0010037422180175781, occupancy inclusion (polygon tiles): 0.0025866031646728516
Number of objects: triangulation: 400, axis-aligned triangulation: 399, oriented rectangles: 107, occupancy inclusion (polygon tiles): 16
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.012439727783203125, number of colliding trajectories: 968, number of collision-free trajectories: 32
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.01119542121887207, number of colliding trajectories: 968, number of collision-free trajectories: 32
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.007790327072143555, number of colliding trajectories: 968, number of collision-free trajectories: 32
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories:

78 DEU_Ffb-1_7_T-1
Pre-processing time: triangulation: 0.04703211784362793, axis-aligned triangulation: 0.0031528472900390625, oriented rectangles: 0.000843048095703125, occupancy inclusion (polygon tiles): 0.010294198989868164
Number of objects: triangulation: 467, axis-aligned triangulation: 434, oriented rectangles: 122, occupancy inclusion (polygon tiles): 39
Triangulation, Uniform grid broadphase; time to check all trajectories: 0.022954702377319336, number of colliding trajectories: 42, number of collision-free trajectories: 958
Axis-aligned Triangulation, Uniform grid broadphase; time to check all trajectories: 0.014455556869506836, number of colliding trajectories: 42, number of collision-free trajectories: 958
Oriented Rectangles, Uniform grid broadphase; time to check all trajectories: 0.01004171371459961, number of colliding trajectories: 42, number of collision-free trajectories: 958
Oriented Rectangles, Box2D dynamic AABB tree broadphase; time to check all trajectories: 0.

### 1.2 Export Data

In [13]:
import csv

with open('scen_static.csv', mode='w') as scen_charact:
    scen_charact_writer = csv.writer(scen_charact, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    column_labels = ['Scenario ID', 
                     'Pre-Processing Time: Triangulation', 'Pre-Processing Time: Axis-aligned Triangulation', 'Pre-Processing Time: Oriented Rectangles', 'Pre-Processing Time: Occupancy Inclusion - Polygon Tiles', 'Pre-Processing Time: Occupancy Inclusion - Lane Polygons', 'Pre-Processing Time: Occupancy Inclusion - whole polygons',
                     'Number of Triangles', 'Number of Axis-aligned Triangles', 'Number of Oriented Rectangles', 'Number of Polygons (polygon tiles)','Number of Polygons (lanes)','Number of Polygons (whole polygons)',
                     'Triangulation: time to check all trajectories', 'Triangulation: number of colliding trajectories', 'Triangulation: number of collision-free trajectories',
                     'Axis-aligned Triangulation: time to check all trajectories', 'Triangulation: number of colliding trajectories', 'Triangulation: number of collision-free trajectories',
                     'Oriented Rectangle with Uniform Grid broadphase: time to check all trajectories', 'Oriented Rectangle with Uniform Grid broadphase: number of colliding trajectories', 'Oriented Rectangle with Uniform Grid broadphase: number of collision-free trajectories',
                     'Oriented Rectangle with Box2D Binary Tree broadphase: time to check all trajectories', 'Oriented Rectangle with Box2D Binary Tree broadphase: number of colliding trajectories', 'Oriented Rectangle with Box2D Binary Tree broadphase: number of collision-free trajectories',
                     'Oriented Rectangle with FCL Binary Tree broadphase: time to check all trajectories', 'Oriented Rectangle with FCL Binary Tree broadphase: number of colliding trajectories', 'Oriented Rectangle with FCL Binary Tree broadphase: number of collision-free trajectories',
                     'Oriented Rectangle with FCL Spatial Hashing broadphase: time to check all trajectories', 'Oriented Rectangle with FCL Spatial Hashing broadphase: number of colliding trajectories', 'Oriented Rectangle with FCL Spatial Hashing broadphase: number of collision-free trajectories',
                     'Oriented Rectangle with FCL Interval Tree broadphase: time to check all trajectories', 'Oriented Rectangle with FCL Interval Tree broadphase: number of colliding trajectories', 'Oriented Rectangle with FCL Interval Tree broadphase: number of collision-free trajectories',
                     'Oriented Rectangle with FCL Sweep-and-Prune broadphase: time to check all trajectories', 'Oriented Rectangle with FCL Sweep-and-Prune broadphase: number of colliding trajectories', 'Oriented Rectangle with FCL Sweep-and-Prune broadphase: number of collision-free trajectories',
                     'Occupancy Inclusion (polygon tiles): time to check all trajectories', 'Occupancy Inclusion: number of colliding trajectories', 'Occupancy Inclusion: number of collision-free trajectories',
                     'Occupancy Inclusion (lanes): time to check all trajectories', 'Occupancy Inclusion: number of colliding trajectories', 'Occupancy Inclusion: number of collision-free trajectories',
                     'Occupancy Inclusion (whole polygon): time to check all trajectories', 'Occupancy Inclusion: number of colliding trajectories', 'Occupancy Inclusion: number of collision-free trajectories',
                    ]
    scen_charact_writer.writerow(column_labels)
    for scenario_id in export_ct.keys():
        if export_res.get(scenario_id, None) is None:
            break
        cur_row = [scenario_id] \
                  + list(export_ct[scenario_id]) \
                  + list(export_obst_num[scenario_id]) \
                  + list(export_res[scenario_id][0]) \
                  + list(export_res[scenario_id][1]) \
                  + list(export_res[scenario_id][2]) \
                  + list(export_res[scenario_id][3]) \
                  + list(export_res[scenario_id][4]) \
                  + list(export_res[scenario_id][5]) \
                  + list(export_res[scenario_id][6]) \
                  + list(export_res[scenario_id][7]) \
                  + list(export_res[scenario_id][8]) \
                  + list(export_res[scenario_id][9]) \
                  + list(export_res[scenario_id][10]) 
        
        scen_charact_writer.writerow(cur_row)
    print('finished')

finished


### 1.3 Error Statistics

In [14]:
tri_tri_errors=0
box_tri_errors=0
poly_tiles_tri_errors=0
poly_lanes_tri_errors=0

total_count=0
for key in export_res.keys():
    el=export_res[key]
    if el[0][1]!=el[1][1]:
        tri_tri_errors=tri_tri_errors+1
    if el[0][1]!=el[2][1]:
        box_tri_errors=box_tri_errors+1
    if el[0][1]!=el[8][1]:
        poly_tiles_tri_errors=poly_tiles_tri_errors+1
    if el[0][1]!=el[9][1]:
        poly_lanes_tri_errors=poly_lanes_tri_errors+1    
    total_count=total_count+1
    
print("Number of scenarios where Triangulation yields different result than Oriented Rectangles (number of colliding trajectories differs): {}".format(box_tri_errors))
print("Number of scenarios where Triangulation yields different result than Axis-aligned Triangulation (number of colliding trajectories differs): {}".format(tri_tri_errors))
print("Number of scenarios where Triangulation yields different result than Occupancy Inclusion - Polygon Tiles (number of colliding trajectories differs): {}".format(poly_tiles_tri_errors))
print("Number of scenarios where Triangulation yields different result than Occupancy Inclusion - Lanes (number of colliding trajectories differs): {}".format(poly_lanes_tri_errors))
print("Number of tested scenarios: {}".format(total_count))

Number of scenarios where Triangulation yields different result than Oriented Rectangles (number of colliding trajectories differs): 0
Number of scenarios where Triangulation yields different result than Axis-aligned Triangulation (number of colliding trajectories differs): 0
Number of scenarios where Triangulation yields different result than Occupancy Inclusion - Polygon Tiles (number of colliding trajectories differs): 0
Number of scenarios where Triangulation yields different result than Occupancy Inclusion - Lanes (number of colliding trajectories differs): 0
Number of tested scenarios: 78


## 2 Collision-Detection in Dynamic Scenarios 

In [15]:
export_res_dyn = dict()

for ind,el in enumerate(os.listdir(directory)[0:]):
    scenario_name = os.path.splitext(el)[0]
    if(True):
        init_list = scenario_init(scenario_name, directory)
        if init_list is None:
            pass
        else:
            scenario, dyn_obstacles, waypoints, trajectories, car_shape, road_boundaries, road_polygons, construction_times = init_list

            if(len(dyn_obstacles)>0):

                print(scenario_name)
                print('Number of dynamic obstacles: '+ str(len(dyn_obstacles)))
                
                res_dynamic=list()
                
                #warming-up
                res_dynamic_uniform_grid = measure_time_dynamic_grid(scenario, trajectories, dyn_obstacles)
                Util.perfStart()
                res_dynamic_uniform_grid = measure_time_dynamic_grid(scenario, trajectories, dyn_obstacles)
                Util.perfReport()
                print('Uniform Grid ' + str(res_dynamic_uniform_grid))
                res_dynamic.append(res_dynamic_uniform_grid)

                res_dynamic_box2d = measure_time_dynamic_box2d(scenario, trajectories, dyn_obstacles)
                Util.perfStart()
                res_dynamic_box2d = measure_time_dynamic_box2d(scenario, trajectories, dyn_obstacles)
                Util.perfReport()
                print('Box2d Tree ' + str(res_dynamic_box2d))
                res_dynamic.append(res_dynamic_box2d)

                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[0,32,1000])
                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[0,32,1000])
                print('FCL Tree ' + str(res_dynamic_fcl))
                res_dynamic.append(res_dynamic_fcl)

                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[1,32,1000])
                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[1,32,1000])
                print('FCl Spatial Hashing ' + str(res_dynamic_fcl))
                res_dynamic.append(res_dynamic_fcl)

                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[2,32,1000])
                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[2,32,1000])
                print('FCL Interval Tree ' + str(res_dynamic_fcl))
                res_dynamic.append(res_dynamic_fcl)

                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[4,32,1000])
                res_dynamic_fcl = measure_time_dynamic_fcl(scenario, trajectories, dyn_obstacles, method=[4,32,1000])
                print('FCL SaP ' + str(res_dynamic_fcl))
                res_dynamic.append(res_dynamic_fcl)


                export_res_dyn[scenario_name]=res_dynamic
            



USA_Peach-2_1_T-1
Number of dynamic obstacles: 16
Uniform Grid (0.009135723114013672, 0, 1000)
Box2d Tree (0.007231473922729492, 0, 1000)
FCL Tree (0.007586479187011719, 0, 1000)
FCl Spatial Hashing (0.016129732131958008, 0, 1000)
FCL Interval Tree (0.01058197021484375, 0, 1000)
FCL SaP (0.008280754089355469, 0, 1000)
USA_Lanker-2_12_T-1
Number of dynamic obstacles: 83
Uniform Grid (0.01058650016784668, 0, 1000)
Box2d Tree (0.012323379516601562, 0, 1000)
FCL Tree (0.013869524002075195, 0, 1000)
FCl Spatial Hashing (0.015503644943237305, 0, 1000)
FCL Interval Tree (0.020494461059570312, 0, 1000)
FCL SaP (0.0199124813079834, 0, 1000)
USA_US101-21_1_T-1
Number of dynamic obstacles: 30
Uniform Grid (0.009500741958618164, 166, 834)
Box2d Tree (0.009248495101928711, 166, 834)
FCL Tree (0.010799646377563477, 166, 834)
FCl Spatial Hashing (0.013185739517211914, 166, 834)
FCL Interval Tree (0.013154983520507812, 166, 834)
FCL SaP (0.010910511016845703, 166, 834)
USA_Lanker-2_11_T-1
Number of dy

CHN_Sha-3_2_T-1
Number of dynamic obstacles: 52
Uniform Grid (0.009568929672241211, 230, 770)
Box2d Tree (0.008123159408569336, 230, 770)
FCL Tree (0.01029825210571289, 230, 770)
FCl Spatial Hashing (0.010794878005981445, 230, 770)
FCL Interval Tree (0.012658357620239258, 230, 770)
FCL SaP (0.015012264251708984, 230, 770)
CHN_Sha-6_4_T-1
Number of dynamic obstacles: 53
Uniform Grid (0.004886627197265625, 956, 44)
Box2d Tree (0.004926443099975586, 956, 44)
FCL Tree (0.008753299713134766, 956, 44)
FCl Spatial Hashing (0.008866310119628906, 956, 44)
FCL Interval Tree (0.012541532516479492, 956, 44)
FCL SaP (0.012324094772338867, 956, 44)
CHN_Sha-14_2_T-1
Number of dynamic obstacles: 98
Uniform Grid (0.01224970817565918, 0, 1000)
Box2d Tree (0.013244867324829102, 0, 1000)
FCL Tree (0.017067432403564453, 0, 1000)
FCl Spatial Hashing (0.017852306365966797, 0, 1000)
FCL Interval Tree (0.02221226692199707, 0, 1000)
FCL SaP (0.03531670570373535, 0, 1000)
CHN_Sha-2_3_T-1
Number of dynamic obstac

CHN_Sha-17_1_T-1
Number of dynamic obstacles: 77
Uniform Grid (0.012147188186645508, 0, 1000)
Box2d Tree (0.014046669006347656, 0, 1000)
FCL Tree (0.01840662956237793, 0, 1000)
FCl Spatial Hashing (0.017928123474121094, 0, 1000)
FCL Interval Tree (0.023003578186035156, 0, 1000)
FCL SaP (0.02364349365234375, 0, 1000)
CHN_Sha-4_6_T-1
Number of dynamic obstacles: 12
Uniform Grid (0.003013134002685547, 992, 8)
Box2d Tree (0.002457141876220703, 992, 8)
FCL Tree (0.0029697418212890625, 992, 8)
FCl Spatial Hashing (0.0036268234252929688, 992, 8)
FCL Interval Tree (0.004011631011962891, 992, 8)
FCL SaP (0.0031015872955322266, 992, 8)
USA_US101-22_2_T-1
Number of dynamic obstacles: 15
Uniform Grid (0.009152889251708984, 196, 804)
Box2d Tree (0.009375810623168945, 196, 804)
FCL Tree (0.010703325271606445, 196, 804)
FCl Spatial Hashing (0.01284480094909668, 196, 804)
FCL Interval Tree (0.011223554611206055, 196, 804)
FCL SaP (0.009680509567260742, 196, 804)
USA_US101-22_1_T-1
Number of dynamic ob

### 2.1 Export Data

In [16]:
import csv
with open('scen_dynamic.csv', mode='w') as scen_dynamic:
    scen_dynamic_writer = csv.writer(scen_dynamic, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    method_labels=['Uniform Grid','Box2D Dynamic Tree', 'FCL Dynamic Tree', 'FCL Spatial Hashing', 'FCL Interval Tree', 'FCL SaP']
    
    column_labels = ['Scenario ID']
    for el in method_labels:
        column_labels.append(el + ' time to check all trajectories')
        column_labels.append(el + ' number of colliding trajectories')
        column_labels.append(el + ' number of collision-free trajectories')
    
    
    scen_dynamic_writer.writerow(column_labels)
    for scenario_id in export_res_dyn.keys():
        cur_row=[scenario_id]
        for ind,el in enumerate(method_labels):
            cur_row = cur_row + list(export_res_dyn[scenario_id][ind])
        
        scen_dynamic_writer.writerow(cur_row)
    print('finished')

finished


### 2.2 Statistics

In [17]:
times_dyn = list()

for el in range(len(method_labels)):
    times_dyn.append(np.zeros(len(export_res_dyn.keys())))

for ind, scenario_id in enumerate(export_res_dyn.keys()):
    new_el = (export_res_dyn[scenario_id], scenario_id)
    for el in range(len(method_labels)):
        times_dyn[el][ind] = export_res_dyn[scenario_id][el][0]
    
for el in range(len(method_labels)):
    print(method_labels[el]+' Mean: ' + str(times_dyn[el].mean()))
for el in range(len(method_labels)):
    print(method_labels[el]+' Std: ' + str(times_dyn[el].std()))
    


Uniform Grid Mean: 0.008746575086544722
Box2D Dynamic Tree Mean: 0.00829006464053423
FCL Dynamic Tree Mean: 0.010119080543518066
FCL Spatial Hashing Mean: 0.012656282155941695
FCL Interval Tree Mean: 0.012606425163073417
FCL SaP Mean: 0.013271206464522924
Uniform Grid Std: 0.0024438032851197395
Box2D Dynamic Tree Std: 0.0031472766596601196
FCL Dynamic Tree Std: 0.0040051254868865675
FCL Spatial Hashing Std: 0.005702305437092174
FCL Interval Tree Std: 0.005059715734317267
FCL SaP Std: 0.008874005518035228
